In [50]:
from pyomo.environ import *
from pyomo.opt import SolverFactory
import numpy as np

# CBE30338 Optimization course
Notes available at: 
- https://jckantor.github.io/CBE30338/06.04-Linear-Production-Model-in-Pyomo.html

###  Example 1: Linear Production Model with Constraints

The mathematical formulation of a simple linear production model is given by

\begin{align}
\max_{x,y \geq 0} &\ 40\ x + 30\ y  & \mbox{objective}\\
\mbox{subject to:}\qquad \\
x & \leq 40  & \mbox{demand constraint} \\
x + y & \leq 80  & \mbox{labor A constraint} \\
2x + y & \leq 100 & \mbox{labor B constraint}
\end{align}

where $x$ and $y$ are the rates of production (in units per week) for two products.

In [8]:
model = ConcreteModel(name="Production")

#Variables
model.x = Var(within=NonNegativeReals, bounds=(0,40))
model.y = Var(within=NonNegativeReals)

#Objective function
model.obj = Objective(expr=40*model.x + 30*model.y, sense = maximize)

#Constraints
def laborA(model):
    return(model.x + model.y <= 80)

def laborB(model):
    return(2*model.x + model.y <= 100)

constraints = [laborA, laborB]

def constraint(model, i):
    return(constraints[i](model))

model.laborA = Constraint([0,1], rule=constraint)

In [12]:
solver = SolverFactory("glpk")
results = solver.solve(model)
print(results)


Problem: 
- Name: unknown
  Lower bound: 2600.0
  Upper bound: 2600.0
  Number of objectives: 1
  Number of constraints: 3
  Number of variables: 3
  Number of nonzeros: 5
  Sense: maximize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.052280426025390625
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [11]:
print(value(model.x), value(model.y))

20.0 60.0


##### Another way of writing the model

In [13]:
model = ConcreteModel(name="Production")

#Variables
model.x = Var(within=NonNegativeReals, bounds=(0,40))
model.y = Var(within=NonNegativeReals)

#Objective function
model.obj = Objective(expr=40*model.x + 30*model.y, sense = maximize)

model.laborA = Constraint(expr=model.x + model.y <= 80)
model.laborB = Constraint(expr=2*model.x + model.y <= 100)

In [17]:
results = SolverFactory("glpk").solve(model)
results.write()
if results.solver.status == "ok":
    model.pprint()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 2600.0
  Upper bound: 2600.0
  Number of objectives: 1
  Number of constraints: 3
  Number of variables: 3
  Number of nonzeros: 5
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.09418249130249023
# ----------------------------------------------------------
#   Solution Information
# -----------------------------------

In [19]:
print('Profit = ', model.obj())

print('\nDecision Variables')
print('x = ', model.x())
print('y = ', model.y())

print('\nConstraints')
print('LaborA = ', model.laborA())
print('LaborB = ', model.laborB())

Profit =  2600.0

Decision Variables
x =  20.0
y =  60.0

Constraints
LaborA =  80.0
LaborB =  100.0


##### Another way of writing the model

In [ ]:
model = ConcreteModel()

solver = SolverFactory('glpk')

# create a model
model = ConcreteModel()

# index for x
idx = ['Product A', 'Product B']

# create decision variables
model.x = Var(idx, domain=NonNegativeIntegers)

# create objective
model.maxprofit = Objective(expr = 40*model.x['Product A'] + 30*model.x['Product B'], sense=maximize)

# create constraints
model.constraints = ConstraintList()
model.constraints.add(model.x['Product A'] <= 40)
model.constraints.add(model.x['Product A'] + model.x['Product B'] <= 80.5)
model.constraints.add(model.x['Product A'] + model.x['Product B'] <= 100)

solver = SolverFactory('glpk')

results = solver.solve(model)

for i in idx:
    print(i, model.x[i]())

##### Another way of writing the model:  Matrix/Vector Format


In [74]:
# We will create two arrays for the constraints
A = np.array([[1, 0], [1, 1],[2,1]])
b = np.array([[40], [80], [100]])
# One array for the objective function
c = np.array([[40, 30]])

# set of row indices
I = range(len(A)) # range starts at 0, excludes last number. 
# set of column indices
J = range(len(A.T)) #count rows, then, we need to Transpose to count columns

# create a model instance
model = ConcreteModel()

# create variables. Since the matrix has two columns (one for x and one for y) we can use the indices of J
model.x = Var(J)

# add model constraints
model.constraints = ConstraintList()
for i in I:
    model.constraints.add(sum(A[i,j]*model.x[j] for j in J) <= b[i,0])

# add a model objective
model.objective = Objective(expr = sum(c[0,j]*model.x[j] for j in J), sense=maximize)

# Create a solver
solver = SolverFactory("glpk")

# solve
solver.solve(model)

# print solutions
for j in J:
    print("x[", j, "] = ", model.x[j].value)
    
model.constraints.display()

x[ 0 ] =  20.0
x[ 1 ] =  60.0
constraints : Size=3
    Key : Lower : Body  : Upper
      1 :  None :  20.0 :  40.0
      2 :  None :  80.0 :  80.0
      3 :  None : 100.0 : 100.0


###  Example 2. Integer. Sum \\$ 83  with minimum number of coins

In [79]:
data = {"penny": 1,
       "nickel": 5,
       "dime": 10,
       "quarter": 25,
       "half-dollar": 50}

model = ConcreteModel()
model.x = Var(data.keys(), domain = NonNegativeIntegers)
model.ncoins = Objective(expr = sum(model.x[c] for c in data.keys()), sense=minimize)
model.cons = Constraint(expr=83 == sum(data[c]*model.x[c] for c in data.keys()))

solver = SolverFactory("glpk")
solver.solve(model)
model.pprint()

1 Set Declarations
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {'dime', 'half-dollar', 'nickel', 'penny', 'quarter'}

1 Var Declarations
    x : Size=5, Index=x_index
        Key         : Lower : Value : Upper : Fixed : Stale : Domain
               dime :     0 :   0.0 :  None : False : False : NonNegativeIntegers
        half-dollar :     0 :   1.0 :  None : False : False : NonNegativeIntegers
             nickel :     0 :   1.0 :  None : False : False : NonNegativeIntegers
              penny :     0 :   3.0 :  None : False : False : NonNegativeIntegers
            quarter :     0 :   1.0 :  None : False : False : NonNegativeIntegers

1 Objective Declarations
    ncoins : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : x[penny] + x[nickel] + x[dime] + x[quarter] + x[half-dollar]

1 Constraint Declarations
    cons : Size=1, Index

In [81]:
data["half-dollar"] + data["nickel"] + data["quarter"] + 3*data["penny"]

83

###  Example 3. Linear Blending Problem  (6.6)

A brewery receives an order for 100 gallons of 4% ABV (alchohol by volume) beer. The brewery has on hand beer A that is 4.5% ABV that cost \\$ 0.32 per gallon to make, and beer B that is 3.7% ABV and cost \\$0.25 per gallon. Water could also be used as a blending agent at a cost of \\$0.05 per gallon. Find the minimum cost blend that meets the customer requirements.